# A Complete Analysis of 283. Move Zeroes (easy)

Question Link: https://leetcode.com/problems/move-zeroes/

## Step 1: Analysis For Bruteforce Solution

This question is fairly straightforward. The requirement is simply move all the zeros in given array ```nums``` to the back of the array.

However, we should take note of the below requirement and constraint: 
- relative order of the non-zero elements in the array should be a maintained. 
- it is not allowed to use a copy of the array. In other words, the space complexity should be O(1).
- input constraints:
    - 1 <= nums.length <= 10**4
    - -231 <= nums[i] <= 231 - 1


For the requirement of in-place alteration, just think if we can set up a new array, then the question becomes a very simple array traversal problem, i.e., loop through the array and copy the non-zero elements in the input array. The time complexity of this algorithm is O(N).

Because the required space complexity is O(1), it complicates the issue. In a typical array data structure, if one element is moved to the end of the array, all elements after it will have to be moved forward to cover the "hole" created by the movement. And this is the bruteforce way of solving this problem.

Combined with the input constraints, we can see that the range of the array length actually causes the bruteforce solution to fail to pass with a TLE (Time Limit Exceeded).

In [8]:
# Solution 1: Bruteforce.
# This solution will not pass because this is an O(N**2))) time complexity.

class Solution:
    def moveZeroes(self, nums: list[int]) -> None:
        """
        Do not return anything, modify nums in-place instead.
        """
        i, cnt0 = 0, 0  # i is the number of non-zero elements so far.
                        # cnt0 is the number of zero elements so far. 
                        # If i + cnt0 == len(nums), no more elements to process
        while i+cnt0 < len(nums):
            if nums[i] == 0:
                cnt0 += 1 # one more 0 found
                for j in range(i+1, len(nums)):
                    nums[j-1] = nums[j] # move every element after pos i forward one position
                nums[-1] = 0 # move the 0 to the end of array
            else:
                i += 1 # one more non-zero element found

In [12]:
# Test solution 1
arrs = [[1, 0, 2, 3, 0, 12], [0], [0, 0, 1]]
for arr in arrs:
    Solution().moveZeroes(arr)
    print(arr)

[1, 2, 3, 12, 0, 0]
[0]
[1, 0, 0]


## Step 2: Analysis of Solution Utilizing the Python List Features

The next solution I can think of is the pop() and append() method provided by the Python list data structure.

The thinking is also very straightforward: use a loop to go through the list and wherever a 0 value is found, use the pop() method to remove the element and append the removed 0 to the end of the list.

However, it is very dangerous to change the elements of a list in a for loop. It will probably create some unexpected result. In our case, if we remove a zero element from the list and append it to the end, because after the removal, the elements after the moved zero will be promoted to the previous positions, some elements will not be processed.

For example, if nums = [0, 1, 2, 3], in the for loop, in the first iteration, the leading zero will be moved to the end, hence nums becomes [1, 2, 3, 0], but in the next iteration, the focus goes to value 2 in position 1, but not the original value 1 in position 1 (since it's moved to position 0).

Here we should use while loop and the zero and non-zero count trick in the bruteforce solution.


In [11]:
# Solution 2: using pop() and append() method from python list
# This solution can pass the test but needs further analysis on time complexity

class Solution:
    def moveZeroes(self, nums: list[int]) -> None:
        """
        Do not return anything, modify nums in-place instead.
        """
        zero = nonzero = 0 # still set two counters zero and nonzero for counting zeros and nonzeros
        while zero + nonzero < len(nums):
            if nums[nonzero] == 0: # it's nonzero here because the first part of the resulting array is nonzero
                nums.append(nums.pop(nonzero)) # using the provided methods
                zero += 1 # one more zero found
            else:
                nonzero += 1 # one more nonzero found

In [14]:
# Test solution 2
arrs = [[1, 0, 2, 3, 0, 12], [0], [0, 0, 1, 0]]
for arr in arrs:
    Solution().moveZeroes(arr)
    print(arr)

[1, 2, 3, 12, 0, 0]
[0]
[1, 0, 0, 0]


### Further Analysis For Solution 2

To determine the time complexity of the solution, we have to look at the time complexity of the pop() and append() method. And here is what I found:
    
https://wiki.python.org/moin/TimeComplexity

Very unfortunately pop(i) is O(N), append() is O(1), and so the time complexity for solution 2 is O(N**2).

##### A question to think about:

If the time complexity for solution 1 and that for 2 are both O(N), why on earth solution 2 can pass the test on Leetcode but solution 1 cannot?


## Step 3: Analysis For Decreasing Time Complexity

Let's look back at the bruteforce solution (solution 1) and try to figure out why the zero and the nonzero counters are used.

As denoted in the comments for solution 1, the two counters are used as sentinels in the while loop. Other than that, the nonzero count is used as a pointer to the position of an element that is be processed in an iteration, as shown below:
```python
    if nums[i] == 0:
        cnt0 += 1 # one more 0 found
        for j in range(i+1, len(nums)):
            nums[j-1] = nums[j] # move every element after pos i forward one position
            nums[-1] = 0 # move the 0 to the end of array
    else:
        i += 1 # one more non-zero element found
```

It is clear the value of the nonzero counter denotes the border of a nonzero value "zone" in the array. It's because when one nonzero element at the index i equal to the nonzero counter is found, the this element is "added" to the nonzero "zone" by increasing the nonzero counter by one.

The zero counter also signals a "zero zone" from the end of the array.

And the elements between those two "zones" are in the "wait zone" waiting to be moved to one of the two "zones" according to their values.

A illustration below:
```
     nonzero zone    <-------   wait zone   -------> zero zone
    +---------------+                               +-------------+
    | ... 2 3 2 8 9 | 2 0 5 8 0 4 0 1 3 9 7 5 6 ... | 0 0 0 0 ... |
    +---------------+                               +-------------+
                      ^ <= nonzero counter           ** size of zero zone is the value of zero counter.
```
The nonzero counter's value is the position (index) of the value being processed in the current iteration.

Then our bruteforce solution becomes:
- loop through the elements in the array
    - if an element is nonzero, increase nonzero counter by 1 to 
        1. increase the size of the nonzero zone.
        2. thus include the element in nonzero zone
    - otherwise (element is zero)
        1. move the element to the back of the array and move all elements after it one position forward.
        2. increase the zero counter by 1 to increase the size of the zero zone.
- Once the nonzero counter steps into the zero zone, the loop should be ended.

In essence, the nonzero and the zero counters separate the given array into three zones and when all the elements in the "wait zone" are processed, we have the desired result.
This picture is much easier to understand than the changing values of the zero and nonzero counters.

--------------

Next, we would like to see whether it is possible to decrease the time complexity of the solution.

In the bruteforce solution, we can quite easily spot the inefficiency. The part that takes most time is the moving of the elements after a zero is found in the "wait zone." In the resulting array, there is a fix position for a nonzero value in the array, i.e., if 5 is the third nonzero value in the input array, its final position (index) should be 2, but in the bruteforce solution, what we do is to move them one position at a time. To address this inefficiency, what we need is a way to move the nonzero values from their positions in the original array directly to the positions in the final array. 

In the current solution, there are two ways we can do to optimize the algorithm. The first is to exchange the zeros in the "wait zone" to the beginning of the "zero zone" and do the movement of the other elements in the "wait zone". This will not work because it reduces the movement number a bit but it does not decrease time complexity to O(N).

The second way is to exchange the zeros with the element right before the first element in the "zero zone." But the resulting array will break the relative order of the nonzero values, failing to meet the first requirement.

Another way is to flip the "wait zone" and the "zero zone," i.e., put the "zero zone" before the "wait zone." In this case, the nonzero counter is pointing to the first element in the "zero zone," and then the value, nonzero counter + zero counter, will be pointing to the first element of the "wait zone." Here let's call the value "front."

Now, if the element at the front index is a zero, we can simple add one to the zero counter to increase the size of "zero zone." And if the element is a nonzero, we can exchange this element with the one pointed by the nonzero counter, which is a zero, and add one to the nonzero counter to increase the size of the "nonzero zone." In this way, if the nonzero counter + the zero counter equals to the length of the array, we can stop the loop.

In this solution, we manage to put the nonzero elements directly to the positions in the resulting array, thus successfully reducing the time complexity to O(N).

Here comes the code.

In [16]:
# solution 3: Flipping "wait zone" and "zero zone"
# The time complexity is O(N)
class Solution:
    def moveZeroes(self, nums: list[int]) -> None:
        """
        Do not return anything, modify nums in-place instead.
        """
        zero, nonzero = 0, 0
        while zero + nonzero < len(nums):
            if nums[zero+nonzero] == 0:
                zero += 1
            else:
                nums[nonzero], nums[zero+nonzero] = nums[zero+nonzero], nums[nonzero]
                nonzero += 1
        


In [17]:
# Test solution 3
arrs = [[1, 0, 2, 3, 0, 12], [0], [0, 0, 1, 0]]
for arr in arrs:
    Solution().moveZeroes(arr)
    print(arr)
        

[1, 2, 3, 12, 0, 0]
[0]
[1, 0, 0, 0]


## Similar Questions:

27. Remove Element: https://leetcode.com/problems/remove-element/

26. Remove Duplicates from Sorted Array: https://leetcode.com/problems/remove-duplicates-from-sorted-array/